# Fruit Network v0.1 - Training Notebook

This is the notebook used for training the FruitNetwork <br>
Batch size should be ajusted to fit in the GPU's memory (if using cuda)

This notebook uses the GPU for most of the work, remove all `.cuda()` to use the CPU (might be slow)

### 1. Importing used libraries

In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils import plot_batch, calculate_loss_and_accuracy
from custom_tranforms import AddRandomBackground, RandomZoom
from FruitModel import FruitModel

### 2. Setting up hyperparameters and transforms

In [2]:
batch_size = 512
learning_rate = 0.001

In [3]:
# Data augmnetation inserted into the network

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation((-45, 45), fill=255),
    RandomZoom((-60, 20)),
    AddRandomBackground((150, 180)),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor()
])

### 3. Loading data (Train and Test set)

In [4]:
validation_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
train_data = ImageFolder('fruits-360/Training', train_transforms)

In [6]:
test_data = ImageFolder('fruits-360/Test', validation_transforms)

In [7]:
train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=0, shuffle=True)

In [8]:
test_loader = DataLoader(test_data, batch_size=batch_size, num_workers=0, shuffle=True)

### 4. Loading the network (and loading preTrained model - optional), criterion and optimizer

In [9]:
model = FruitModel()
model.load_state_dict(torch.load('models/fruit_bg_net.pt')['state_dict'])
model = model.cuda()

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

### 5. Training

In [12]:
def train(epochs):
    """
        Function to train the network for {epoch} epochs.
        Trains data in minibatches, and after each full epoch calculates
        the loss and accuracy for the train and test dataset

    Arguments:
        epochs {int} -- number of epochs to be trained

    Returns:
        acc {float} -- last computed validation (test) accuracy
    """
    
    print("Trainning is about to start...")

    # Prints loss 5 times per epoch
    print_every = len(train_loader)//5

    for epoch in range(1, epochs+1):
        print('---- EPOCH {} ----'.format(epoch))
        step = 0
        for i, (inputs, labels) in enumerate(train_loader):
            
            model.train()

            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            step += 1

            if step % print_every == 0:
                print('[{}] Step {}: Loss: {}'.format(epoch, step, loss))
        model.eval()

        _, train_acc = calculate_loss_and_accuracy(train_loader, model, criterion)
        _, test_acc = calculate_loss_and_accuracy(test_loader, model, criterion)
        print('Epoch {} - Train Acc: {:.2f} Validation Acc: {:.2f}'.format(epoch, train_acc, test_acc))

    return test_acc

In [17]:
learning_rate = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
acc = train(30)
learning_rate = learning_rate / 10
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
acc = train(30)

Trainning is about to start...
---- EPOCH 1 ----
[1] Step 100: Loss: 0.5863775014877319
Epoch 1 - Train Acc: 91.47 Validation Acc: 94.47
---- EPOCH 2 ----
[2] Step 100: Loss: 0.5792826414108276
Epoch 2 - Train Acc: 91.08 Validation Acc: 95.57
---- EPOCH 3 ----
[3] Step 100: Loss: 0.6074717044830322
Epoch 3 - Train Acc: 90.56 Validation Acc: 94.86
---- EPOCH 4 ----
[4] Step 100: Loss: 0.5511271357536316
Epoch 4 - Train Acc: 90.69 Validation Acc: 94.27
---- EPOCH 5 ----
[5] Step 100: Loss: 0.6116474866867065
Epoch 5 - Train Acc: 91.15 Validation Acc: 95.51
---- EPOCH 6 ----
[6] Step 100: Loss: 0.6005362868309021
Epoch 6 - Train Acc: 91.15 Validation Acc: 94.47
---- EPOCH 7 ----
[7] Step 100: Loss: 0.5901138782501221
Epoch 7 - Train Acc: 91.02 Validation Acc: 95.31
---- EPOCH 8 ----
[8] Step 100: Loss: 0.5696591138839722
Epoch 8 - Train Acc: 89.32 Validation Acc: 95.57
---- EPOCH 9 ----
[9] Step 100: Loss: 0.5146848559379578
Epoch 9 - Train Acc: 91.93 Validation Acc: 95.64
---- EPOCH 10 -

### 6. Saving the trained network

In [19]:
torch.save({'state_dict':model.state_dict(), 'acc': acc}, 'models/fruit_bg_net.pt')